# HuggingFace로 두 문장의 논리적 모순 분류하기

이번 실습에서는 HuggingFace로 영화리뷰 감정 분석과 같은 text 분류 문제를 위한 모델을 구현합니다.
먼저 필요한 library들을 설치하고 import합니다.

In [52]:
!pip install transformers datasets evaluate accelerate scikit-learn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [53]:
import random
import evaluate
import numpy as np

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

## [MY CODE]Dataset 준비

그 다음 논리적 모순 분류을 위해 사용할 MNLI dataset을 `load_dataset` 함수로 다운로드 받습니다.

In [149]:
ds = load_dataset("nyu-mll/glue", "mnli")

# 유니크한 라벨 수 구하기
unique_labels = set(ds['train']['label'])
num_unique_labels = len(unique_labels)

print(ds)
print(f"유니크한 라벨 수: {num_unique_labels}")

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})
유니크한 라벨 수: 3


`load_dataset`은 HuggingFace의 `datasets` library의 함수로, HuggingFace의 hub에서 dataset을 다운로드 받을 수 있도록 만든 함수입니다.
출력 결과를 보시면 `mnli`는 `train`, `validation_matched`, 그리고 `validation_mismatched`, `test_matched`, `test_mismatched` data로 구성되어있습니다.
이 중에서 우리는 `train`과 `validation_matched`를 활용합니다.

`train` data를 한 번 살펴보겠습니다.

In [150]:
ds['train'][0]

{'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.',
 'hypothesis': 'Product and geography are what make cream skimming work. ',
 'label': 1,
 'idx': 0}

## [MY CODE]
`train`과 `validation_matched`의 각 data는 `premise`와 `hypothesis`, `label`로 구성되어있습니다.
각각은 두문장과 해당 문장의 논리적 모순점 여부를 의미합니다.

이번에는 tokenizer를 불러와서 미리 text들을 tokenize합니다.
두 문장을 파라미터로 전달하여 [SEP] 토큰을 매개체로 이어지게 구성합니다.

In [151]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def preprocess_function(data):
    return tokenizer(data["premise"], data["hypothesis"], truncation=True)

ds_tokenized = ds.map(preprocess_function, batched=True)

Map: 100%|██████████| 9847/9847 [00:00<00:00, 28906.94 examples/s]


Tokenizer를 실행할 때 넘겨주었던 `truncation` 옵션은 주어진 text가 일정 길이 이상이면 잘라내라는 의미입니다.
만약 특정 길이 값이 같이 주어지지 않는다면 `bert-base-cased`를 학습할 때 사용한 text의 최대 길이를 기준으로 값을 결정합니다.

In [152]:
ds_tokenized['train'][0].keys()

dict_keys(['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'])

마지막 출력 결과를 보면, `text`와 `label` 이외에 `input_ids`가 생기신 것을 확인하실 수 있습니다.
이는 우리가 `AutoTokenizer.from_pretrained`로 불러온 tokenizer로 text를 token들로 나누고 정수 index로 변환한 결과입니다.

이번에는 `train` data를 쪼개 training data와 validation data를 만들어보겠습니다.

In [153]:
ds_split = ds_tokenized['train'].train_test_split(test_size=0.2)
ds_train, ds_val = ds_split['train'], ds_split['test']
ds_test = ds_tokenized['validation_matched']

print(ds_train[0])
print(ds_val[0])

{'premise': "really oh that's the way it is up here like today it was fifty and raining and two days ago it was snowing a blizzard up here so  it just it just likes kind of crazy all over the country i guess", 'hypothesis': 'The weather has been stable over the past few days.', 'label': 2, 'idx': 323019, 'input_ids': [101, 1541, 9294, 1115, 112, 188, 1103, 1236, 1122, 1110, 1146, 1303, 1176, 2052, 1122, 1108, 5547, 1105, 4458, 1158, 1105, 1160, 1552, 2403, 1122, 1108, 4883, 1158, 170, 171, 24002, 1146, 1303, 1177, 1122, 1198, 1122, 1198, 7407, 1912, 1104, 4523, 1155, 1166, 1103, 1583, 178, 3319, 102, 1109, 4250, 1144, 1151, 6111, 1166, 1103, 1763, 1374, 1552, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

HuggingFace `datasets`로 불러온 dataset은 `train_test_split`으로 쉽게 쪼갤 수 있습니다.

다음은 각 split의 크기입니다.

In [154]:
len(ds_train), len(ds_val), len(ds_test)

(314161, 78541, 9815)

## [MY CODE] Model 구현

이번에는 text 분류를 수행할 Transformer를 구현합니다.
이전에는 Transformer의 구성 요소들을 직접 구현하여 합쳤습니다.
이번에는 HuggingFace의 BERT를 활용하여 인자만 넘겨주는 식으로 구현해보겠습니다:

hidden dimension을 기존 768 -> 128 차원으로, intermediate_size 역시 128차원으로 구성합니다.

In [163]:
from transformers import BertConfig

config = BertConfig()

config.hidden_size = 128  # BERT layer의 기본 hidden dimension
config.intermediate_size = 128  # FFN layer의 중간 hidden dimension
config.num_hidden_layers = 2  # BERT layer의 개수
config.num_attention_heads = 4  # Multi-head attention에서 사용하는 head 개수
config.num_labels = 3  # 마지막에 예측해야 하는 분류 문제의 class 개수

model = AutoModelForSequenceClassification.from_config(config)

print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

BERT는 이전에 배운 Transformer의 architecture를 그대로 사용합니다.
그래서 BERT의 옵션들만 수정하면 vanilla Transformer를 쉽게 구현할 수 있습니다.

Transformer 구현 이외에 분류 문제에 맞춰 첫 번째 token을 linear classifier를 거치는 등의 과정은 `AutoModelForSequenceClassification`이 구현해줍니다.
즉, 우리가 `config`로 넘겨주는 BERT의 마지막에 linear classifier를 달아주는 역할을 합니다.

## [MY CODE] 학습 코드

다음은 위에서 구현한 Transformer를 MNLI 학습하는 코드를 구현합니다.
먼저 다음과 같이 학습 인자들을 정의합니다.

In [164]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='hf_transformer',  # 모델, log 등을 저장할 directory
    num_train_epochs=10,  # epoch 수
    per_device_train_batch_size=64,  # training data의 batch size
    per_device_eval_batch_size=64,  # validation data의 batch size
    logging_strategy="epoch",  # Epoch가 끝날 때마다 training loss 등을 log하라는 의미
    do_train=True,  # 학습을 진행하겠다는 의미
    do_eval=True,  # 학습 중간에 validation data에 대한 평가를 수행하겠다는 의미
    eval_strategy="epoch",  # 매 epoch가 끝날 때마다 validation data에 대한 평가를 수행한다는 의미
    save_strategy="epoch",  # 매 epoch가 끝날 때마다 모델을 저장하겠다는 의미
    learning_rate=2e-5,  # optimizer에 사용할 learning rate
    load_best_model_at_end=True  # 학습이 끝난 후, validation data에 대한 성능이 가장 좋은 모델을 채택하겠다는 의미
)

각각의 부분들은 이전 주차에서 배웠던 내용들을 설정하는 것에 불과하다는 것을 알 수 있습니다.
요약하면 다음과 같습니다:
- `epochs`: training data를 몇 번 반복할 것인지 결정합니다.
- `batch_size`: training data를 얼마나 잘게 잘라서 학습할 것인지 결정합니다.
- `learning_rate`: optimizer의 learning rate를 얼마로 할 것인지 결정합니다.
위의 부분들 이외에도 사소한 구현 요소들도 지정할 수 있습니다.

다음은 loss 이외의 평가 함수들을 구현하는 방법입니다.

In [165]:
import evaluate

accuracy = evaluate.load("accuracy")


def compute_metrics(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

`evaluate` 또한 HuggingFace의 library로 다양한 평가 함수들을 제공하고 있습니다.
이번 실습의 경우, 논리 오류를 찾는 문제는 분류 문제이기 때문에 정확도를 계산할 수 있습니다.
위와 같이 예측 결과(`pred`)와 실제 label(`labels`)가 주어졌을 때 정확도를 계산하는 것은 `evaluate`의 accuracy 함수로 구현할 수 있습니다.

마지막으로 위의 요소들을 종합하여 학습할 수 있는 `Trainer`를 구현합니다.

In [166]:
from transformers import TrainerCallback

# Trainer에 Callback 추가
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    # callbacks=[LoggingCallback()],
)

/var/folders/mf/y70ln4v506xd0nsz7zkn20x40000gn/T/ipykernel_39161/293631768.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


모델, training 인자, training과 validation data, 부가적인 평가 함수, 그리고 tokenizer를 넘겨주면 끝입니다.

위와 같이 만든 `Trainer`는 다음과 같이 학습을 할 수 있습니다.

In [168]:
trainer.train()

 10%|█         | 4909/49090 [07:58<52:23, 14.06it/s]  

{'loss': 1.0642, 'grad_norm': 1.8437665700912476, 'learning_rate': 1.8e-05, 'epoch': 1.0}


 10%|█         | 4909/49090 [08:47<52:23, 14.06it/s]

{'eval_loss': 1.0247308015823364, 'eval_accuracy': 0.4713334436790975, 'eval_runtime': 48.821, 'eval_samples_per_second': 1608.753, 'eval_steps_per_second': 25.153, 'epoch': 1.0}


 20%|██        | 9818/49090 [14:59<1:47:33,  6.09it/s] 

...

100%|██████████| 49090/49090 [1:09:54<00:00,  3.75it/s]   

{'loss': 0.8869, 'grad_norm': 9.048386573791504, 'learning_rate': 0.0, 'epoch': 10.0}



100%|██████████| 49090/49090 [1:10:34<00:00, 11.59it/s]

{'eval_loss': 0.9441840052604675, 'eval_accuracy': 0.5496746921989789, 'eval_runtime': 40.0177, 'eval_samples_per_second': 1962.657, 'eval_steps_per_second': 30.686, 'epoch': 10.0}
{'train_runtime': 4234.2477, 'train_samples_per_second': 741.952, 'train_steps_per_second': 11.594, 'train_loss': 0.939588021522968, 'epoch': 10.0}


TrainOutput(global_step=49090, training_loss=0.939588021522968, metrics={'train_runtime': 4234.2477, 'train_samples_per_second': 741.952, 'train_steps_per_second': 11.594, 'total_flos': 446876333509692.0, 'train_loss': 0.939588021522968, 'epoch': 10.0})

보시다시피 training loss도 잘 떨어졌고, eval_accuracy 역시 잘 떨어지는걸 볼수 있습니다.

위와 같이 학습이 끝난 후 validation loss가 가장 낮은 모델을 가지고 test data의 성능을 평가하는 것은 다음과 같이 구현할 수 있습니다.

In [169]:
trainer.evaluate(ds_test)

100%|██████████| 154/154 [00:06<00:00, 23.53it/s]


{'eval_loss': 0.9302733540534973,
 'eval_accuracy': 0.5644421803362201,
 'eval_runtime': 6.9348,
 'eval_samples_per_second': 1415.317,
 'eval_steps_per_second': 22.207,
 'epoch': 10.0}

이전에 학습 인자에서 `load_best_model_at_end=True`를 넘겨줬기 때문에 `trainer`는 학습이 끝난 후, 기본적으로 validation loss가 가장 좋은 모델을 가지고 `evaluate`를 진행합니다.
실제로 결과를 보면 `eval_loss`가 가장 낮은 validation loss와 유사한 것을 볼 수 있습니다.

평가할 때 사용한 모델은 다음과 같이 저장할 수 있습니다.

In [170]:
trainer.save_model()